# Example of a pretrained RNN charakter model. 

This example show a pre-trained RNN making predictions. 

#### Data and preprocessing
The text for training has been extracted for the Phd thesis:

Joller-Graf, Klaus, Herrn Prof Dr Wilfried Schley, and Frau Prof Dr Ingeborg Kriwet. "Didaktik des integrativen Unterrichts."

Which can be downloaded from: http://edudoc.ch/record/3408/files/zu05056.pdf

Creating the text-file from the pdf via:
```
    ~/Downloads/xpdfbin-mac-3.04/bin64/pdftotext  -enc UTF-8 zu05056.pdf
```
http://stackoverflow.com/questions/4039930/how-to-save-text-file-in-utf-8-format-using-pdftotext

This text has been preprocessed with 
```
X, Y, char_idx = \
    textfile_to_semi_redundant_sequences(path, seq_maxlen=maxlen, redun_step=3)
```
yielding
```
Text total length: 1418267
Distinct chars: 109
Total sequences: 472748
```

### The network
Here we use the trained network, which we obtained using the library tflearn, as follows:

#### Definition
The network has been defined in tflearn as:
```
g = tflearn.input_data([None, maxlen, len(char_idx)])
g = tflearn.lstm(g, 512, return_seq=True)
g = tflearn.dropout(g, 0.5)
g = tflearn.lstm(g, 512, return_seq=True)
g = tflearn.dropout(g, 0.5)
g = tflearn.lstm(g, 512)
g = tflearn.dropout(g, 0.5)
g = tflearn.fully_connected(g, len(char_idx), activation='softmax')
g = tflearn.regression(g, optimizer='ADAM', loss='categorical_crossentropy',
                       learning_rate=0.001)

m = tflearn.SequenceGenerator(g, dictionary=char_idx,
                              seq_maxlen=maxlen,
                              clip_gradients=5.0,
                              checkpoint_path='model_shakespeare')
```
Note that the network guesses the next letter using all, the hidden states of all neurons of the last layer. 

#### Training
...and fitted (50 epochs) using:
```
for i in range(50):
        seed = random_sequence_from_textfile(path, maxlen)
        m.fit(X, Y, validation_set=0.1, batch_size=128,
              n_epoch=1, run_id='shakespeare')
        print("-- TESTING...")
        print("-- Test with temperature of 1.0 --")
        print(m.generate(600, temperature=1.0, seq_seed=seed))
        print("-- Test with temperature of 0.5 --")
        print(m.generate(600, temperature=0.5, seq_seed=seed))    
```

#### Freezing
finally the model has been frozen with:
```
# Loading the pretrained model from the checkpoint
m.load('/home/dueo/Dropbox/__ZHAW/Projekte/RNN/model_shakespeare-166250')
sess = m.session
graph = tf.get_default_graph()
input_graph_def = graph.as_graph_def()
from tensorflow.python.framework import graph_util
# The output node names are used to determine which 
# part of the graph needs to be frozen.
output_node_names = "FullyConnected/Softmax"
output_graph_def = graph_util.convert_variables_to_constants(
        sess, # The session is used to retrieve the weights
        input_graph_def, # The graph_def is used to retrieve the nodes
        output_node_names.split(",") 
    )

with tf.gfile.GFile('didactic_25.pb', "wb") as f:
    f.write(output_graph_def.SerializeToString())
```

In [1]:
import tensorflow as tf
from __future__ import print_function
import os
import pickle
import numpy as np

The network ..

In [2]:
maxlen = 25 #The maximal lenth of the sequence

In [3]:
with open('zu05056_char_idx.pkl', 'rb') as f:
    char_idx = pickle.load(f)
char_idx.keys()[20:30],char_idx.values()[20:30]

(['l', 'p', 't', 'x', '\x80', '\x84', '\x90', '\x98', '\x9c', '\xa0'],
 [20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

### Downloading of the network

In [4]:
# Downloading the model, if it does not exist
import urllib
import os
if not os.path.isfile('didactic_25.pb'):
    urllib.urlretrieve("https://dl.dropboxusercontent.com/u/9154523/models/rnn_fun/didactic_25.pb", "didactic_25.pb")
%ls -hl didactic_25.pb

-rw-r--r--  1 oli  staff    21M Jan  9 11:01 didactic_25.pb


In [5]:
with tf.gfile.GFile('didactic_25.pb', "rb") as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    
tf.reset_default_graph()
graph = tf.Graph().as_default() 
tf.import_graph_def(graph_def,  name='')

In [6]:
#ops = tf.get_default_graph().get_operations()
#for i in ops:print(i.name)

In [7]:
graph = tf.get_default_graph()
feed = graph.get_tensor_by_name('InputData/X:0')
fetch = graph.get_tensor_by_name('FullyConnected/Softmax:0')

In [18]:
# The seed for prediction. Note that the seed need to be exactely of length maxlen
seed = 'Die Grundlagen war dabei '[0:maxlen]
seed, len(seed)

('Die Grundlagen war dabei ', 25)

In [19]:
# Creating a one-hot-encoded matrix
X = np.zeros((1, maxlen, len(char_idx))) #One Batch, t, X_t (one-got-encoded)
for t, char in enumerate(seed):
    X[0, t, char_idx[char]] = 1.0  

In [20]:
with tf.Session() as sess:
    pred = sess.run(fetch, feed_dict={feed:X})

In [29]:
nl = np.argmax(pred) #next letter
nl

18

In [30]:
# Code taken from from tflearn
def reverse_dictionary(char_idx):
    # Build reverse dict
    rev_dic = {}
    for key in char_idx:
        rev_dic[char_idx[key]] = key
    return rev_dic

rev_dic = reverse_dictionary(char_idx)
rev_dic[nl]

'd'

In [35]:
def _sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    logit = np.log(a) 
    p = np.exp(logit / temperature)
    #1.001 to be on the save side, sum(p) < 1 for np.random.multinomial
    p /= (1.001 * np.sum(p))
    return np.argmax(np.random.multinomial(1, p, 1))

n = _sample(pred[0])
n, rev_dic[n]

(18, 'd')

In [37]:
import sys
# Code adapted from tflearn SequenceGenerator
def generate(sess, seq_seed, show=True, seq_length = 400, temperature = 0.1,  seq_maxlen=25):
    sequence = seq_seed
    generated = seq_seed
    dic = char_idx
    rev_dic = reverse_dictionary(dic)


    whole_sequence = seq_seed

    for i in range(seq_length):
        X = np.zeros((1, seq_maxlen, len(dic)))
        for t, char in enumerate(sequence):
            X[0, t, dic[char]] = 1.
        preds = sess.run(fetch, feed_dict={feed:X})[0] #Getting next letter distribution
        next_index = _sample(preds, temperature) #Sampling a letter from the distribution
        #next_index = np.argmax(preds)
        next_char = rev_dic[next_index]
        if show:
            sys.stdout.write(next_char)
            sys.stdout.flush()
        generated += next_char
        sequence = sequence[1:] + next_char
        whole_sequence += next_char
    return whole_sequence

with tf.Session() as sess:
    res = generate(sess, seed, temperature=1.0)
    print('\n')
    print(res)

indivi

/Library/Python/2.7/site-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


duell nach Berufs vereinfachen. Dabei geht es darum, Biete. Ich spezifische Grundlage zwischen verschiedenen Aufbau nehmen, das Lernkeln in einer schwierigen Lernzielen und Externen oder moder sind wichtig ist: ���Integrativ unterrichten der Basis zur��ckbereiten k��nnen, es geht ein Lernst��rungen, sondern auch hier, dass die Fragestellung eingeschultert. Du geht jem��sste einzelne Sch��ler

Die Grundlagen war dabei individuell nach Berufs vereinfachen. Dabei geht es darum, Biete. Ich spezifische Grundlage zwischen verschiedenen Aufbau nehmen, das Lernkeln in einer schwierigen Lernzielen und Externen oder moder sind wichtig ist: „Integrativ unterrichten der Basis zurückbereiten können, es geht ein Lernstörungen, sondern auch hier, dass die Fragestellung eingeschultert. Du geht jemüsste einzelne Schüler


### Some further examples

In [16]:
ts = (1.0, 1.0, 0.5, 0.5, 0.1, 0.05)
with tf.Session() as sess:
    for t in ts:
        print()
        print("Temperature {}".format(t))
        print(generate(sess, seed, temperature=t, show=False))


Temperature 1.0


/Library/Python/2.7/site-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


Die Grundlagen war dabei nach abschliessen. „In der eigene Prozesses Lerngebundents in den Interviews jegentliche Planungsprobleme dadurch, dass die Kinder lässt sich Akzeptanz. Das ist erhöht den Bedeutung nötige Delitivität bringen, die Stofferfachung oder dem Kerngegenständen zur Forschungs- und Zeit“ (Hanna Z./MS2).
319

Didaktik des integrativen Unterrichts
Überlegungen für Integration in die Rolle von As

Temperature 1.0
Die Grundlagen war dabei die Weiterbildung ein zu verhalten, bzw. bei Kinder und Jugendliche neaber lernbehinderter und Lehr- und Lehren (‚Abers für Analyse: Eines der gesellschaftlichenden sich ableiten zu führen. „Wir schlechten Experten sollte nicht mehr anrizerieren. Sie müssen die Monoten begegnet sich es „geübt (vgl. Abb. 4.7) aus verschiedenen Problemen. Wir hätten wir mit aufgesetzt werden nie für den 

Temperature 0.5
Die Grundlagen war dabei die Aussage, dass die Verantwortung und gewissermassen die Lehrperson die Lehrpersonen sehr unterschiedlichen Be

#### Some observations:

* For low temperatures, the system gets trapped in a kind of local minima.
* At least it's political correct:'Schülerinnen und Schüler, 'Heilpädagogin oder dem Heilpädagogen'